In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json, argparse, os
import re
import io
import sys
from keras.models import Model
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import num2words
import gensim
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import enchant
from enchant.checker import SpellChecker
from nltk.metrics.distance import edit_distance

/Users/tchen/homebrew/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ImportError: No module named num2words

In [ ]:
numDimensions = 300
maxSeqLength = 250
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000

In [ ]:
contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have",
"don’t": "do not",
"i'm": "i am"
}

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'l", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\'em", " them", phrase)
    phrase = re.sub(r"\'nt", " not", phrase)
    phrase = re.sub(r"\'", " ", phrase)
    
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’l", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    phrase = re.sub(r"\’em", " them", phrase)
    phrase = re.sub(r"\’nt", " not", phrase)
    phrase = re.sub(r"\’", " ", phrase)
    
    
    return phrase

In [ ]:
class MySpellChecker():

    def __init__(self, dict_name='en_US', max_dist=2):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = max_dist

    def replace(self, word):
        suggestions = self.spell_dict.suggest(word)

        if suggestions:
            for suggestion in suggestions:
                if edit_distance(word, suggestion) <= self.max_dist:
                    return suggestions[0]

        return word

In [3]:
label2emotion = {0:"others", 1:"happy", 2: "sad", 3:"angry"}
emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}
def preprocessData(dataFilePath, mode):
    """Load data from a file, process and return indices, conversations and labels in separate lists
    Input:
        dataFilePath : Path to train/test file to be processed
        mode : "train" mode returns labels. "test" mode doesn't return labels.
    Output:
        indices : Unique conversation ID list
        conversations : List of 3 turn conversations, processed and each turn separated by the <eos> tag
        labels : [Only available in "train" mode] List of labels
    """

    with open('emoji/emoji_ranks.json', 'r') as fn:
        e_data = json.load(fn)
    
    pos_emoticons = e_data['pos']
    neg_emoticons = e_data['neg']
    neutral_emoticons = e_data['neu']
        
    # Emails
    emailsRegex=re.compile(r'[\w\.-]+@[\w\.-]+')

    # Mentions
    userMentionsRegex=re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)')

    #Urls
    urlsRegex=re.compile(r'(f|ht)(tp)(s?)(://)(.*)[.|/][^ ]+') # It may not be handling all the cases like t.co without http

    #Numerics
    numsRegex=re.compile(r"\b\d+\b")

    punctuationNotEmoticonsRegex=re.compile(r'([!?.,-]){2,}')
    
    elongatedWords = re.compile(r'\b(\S*?)(.)\2{2,}\b')
    allCaps = re.compile(r"((?![<]*}) [A-Z][A-Z]+)")

    emoticonsDict = {}
    for i,each in enumerate(pos_emoticons):
        emoticonsDict[each]= ' <SMILE> '
    for i,each in enumerate(neg_emoticons):
        emoticonsDict[each]=' <SADFACE> '
    for i,each in enumerate(neutral_emoticons):
        emoticonsDict[each]=' <NEUTRALFACE> '
    # use these three lines to do the replacement
    rep = dict((re.escape(k), v) for k, v in emoticonsDict.items())
    emoticonsPattern = re.compile("|".join(rep.keys()))
    indices = []
    conversations = []
    labels = []
    u1 = []
    u2 = []
    u3 = []
    indices = []
#     my_spell_checker = MySpellChecker(max_dist=1)
    df = pd.read_csv('Sentiment-Analysis-Dataset.csv', sep=',')
    sentences = df.SentimentText.values
    sentiment = df.Sentiment.values
    
    for label, line in zip(sentiment,sentences):
        # Convert multiple instances of . ? ! , to single instance
        # okay...sure -> okay . sure
        # okay???sure -> okay ? sure
        # Add whitespace around such punctuation
        # okay!sure -> okay ! sure
#             repeatedChars = ['.', '?', '!', ',']

        line = emoticonsPattern.sub(lambda m: rep[re.escape(m.group(0))], line.strip())
        line = userMentionsRegex.sub(' <USER> ', line )
        line = emailsRegex.sub(' <EMAIL> ', line )
        line = urlsRegex.sub(' <URL> ', line)
        line = numsRegex.sub(' <NUMBER> ',line)
        line = punctuationNotEmoticonsRegex.sub(r' \1 <REPEAT> ',line)
        line = elongatedWords.sub(r'\1\2 <ELONG> ', line)
        line = allCaps.sub(r'\1 <ALLCAPS> ', line)
        line = re.sub('([.,!?])', r' \1 ', line)
        line = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", r" <NUMBER> ", line)
        line = re.sub(r'(.)\1{2,}', r'\1\1',line)
#             chkr = SpellChecker("en_US", line)
#             for err in chkr:
#                 err.replace(my_spell_checker.replace(err.word))
#             line = chkr.get_text()

        if mode == "train":
            # Train data contains id, 3 turns and label
            labels.append(label)

        # Remove any duplicate spaces
        duplicateSpacePattern = re.compile(r'\ +')
        conv = re.sub(duplicateSpacePattern, ' ', line)
        conversations.append(line.lower())
    
    if mode == "train":
        return  labels, conversations
    else:
        return indices, conversations, u1, u2, u3

In [4]:
def getMetrics(predictions, ground):
    """Given predicted labels and the respective ground truth labels, display some metrics
    Input: shape [# of samples, NUM_CLASSES]
        predictions : Model output. Every row has 4 decimal values, with the highest belonging to the predicted class
        ground : Ground truth labels, converted to one-hot encodings. A sample belonging to Happy class will be [0, 1, 0, 0]
    Output:
        accuracy : Average accuracy
        microPrecision : Precision calculated on a micro level. Ref - https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/16001
        microRecall : Recall calculated on a micro level
        microF1 : Harmonic mean of microPrecision and microRecall. Higher value implies better classification  
    """
    # [0.1, 0.3 , 0.2, 0.1] -> [0, 1, 0, 0]
    discretePredictions = to_categorical(predictions.argmax(axis=1))
    
    truePositives = np.sum(discretePredictions*ground, axis=0)
    falsePositives = np.sum(np.clip(discretePredictions - ground, 0, 1), axis=0)
    falseNegatives = np.sum(np.clip(ground-discretePredictions, 0, 1), axis=0)
    
    print("True Positives per class : ", truePositives)
    print("False Positives per class : ", falsePositives)
    print("False Negatives per class : ", falseNegatives)
    
    # ------------- Macro level calculation ---------------
    macroPrecision = 0
    macroRecall = 0
    # We ignore the "Others" class during the calculation of Precision, Recall and F1
    for c in range(1, NUM_CLASSES):
        precision = truePositives[c] / (truePositives[c] + falsePositives[c])
        macroPrecision += precision
        recall = truePositives[c] / (truePositives[c] + falseNegatives[c])
        macroRecall += recall
        f1 = ( 2 * recall * precision ) / (precision + recall) if (precision+recall) > 0 else 0
        print("Class %s : Precision : %.3f, Recall : %.3f, F1 : %.3f" % (label2emotion[c], precision, recall, f1))
    
    macroPrecision /= 3
    macroRecall /= 3
    macroF1 = (2 * macroRecall * macroPrecision ) / (macroPrecision + macroRecall) if (macroPrecision+macroRecall) > 0 else 0
    print("Ignoring the Others class, Macro Precision : %.4f, Macro Recall : %.4f, Macro F1 : %.4f" % (macroPrecision, macroRecall, macroF1))   
    
    # ------------- Micro level calculation ---------------
    truePositives = truePositives[1:].sum()
    falsePositives = falsePositives[1:].sum()
    falseNegatives = falseNegatives[1:].sum()    
    
    print("Ignoring the Others class, Micro TP : %d, FP : %d, FN : %d" % (truePositives, falsePositives, falseNegatives))
    
    microPrecision = truePositives / (truePositives + falsePositives)
    microRecall = truePositives / (truePositives + falseNegatives)
    
    microF1 = ( 2 * microRecall * microPrecision ) / (microPrecision + microRecall) if (microPrecision+microRecall) > 0 else 0
    # -----------------------------------------------------
    
    predictions = predictions.argmax(axis=1)
    ground = ground.argmax(axis=1)
    accuracy = np.mean(predictions==ground)
    
    print("Accuracy : %.4f, Micro Precision : %.4f, Micro Recall : %.4f, Micro F1 : %.4f" % (accuracy, microPrecision, microRecall, microF1))
    return accuracy, microPrecision, microRecall, microF1

In [5]:
def getEmbeddingMatrix(wordIndex, out_of_vocab):
    """Populate an embedding matrix using a word-index. If the word "happy" has an index 19,
       the 19th row in the embedding matrix should contain the embedding vector for the word "happy".
    Input:
        wordIndex : A dictionary of (word : index) pairs, extracted using a tokeniser
    Output:
        embeddingMatrix : A matrix where every row has 100 dimensional GloVe embedding
    """
    embeddingsIndex = {}
    
    # Load the embedding vectors from ther GloVe file #glove.twitter.27B.100d.txt #glove.840B.300d.txt #glove.twitter.emoji.100d.txt
    #with io.open(os.path.join(gloveDir, 'glove.twitter.emoji.100d.txt'), encoding="utf8") as f:
    vocab = []
    with io.open(gloveDir, encoding="utf8") as f:
        for line in f:
            values = line.split(' ')
           # print(values)
            word = values[0]
            vocab.append(word)
            embeddingVector = np.array([float(val) for val in values[1:]])
            embeddingsIndex[word] = embeddingVector
    
    print('Found %s word vectors.' % len(embeddingsIndex))
    
    #model = gensim.models.KeyedVectors.load_word2vec_format(gloveDir, binary=False)
    #vocab = model.vocab.keys()

    # Minimum word index of any word is 1. 
    embeddingMatrix = np.zeros((len(wordIndex) + 1, EMBEDDING_DIM))
    for word, i in wordIndex.items():
        embeddingVector = embeddingsIndex.get(word)
        if word not in vocab:
            out_of_vocab.append(word)
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector
    
    return embeddingMatrix, embeddingsIndex

In [6]:
# parser = argparse.ArgumentParser(description="Baseline Script for SemEval")
# parser.add_argument('-config', help='Config to read details', required=True)
# args = parser.parse_args()

with open('testBaseline.config') as configfile:
    config = json.load(configfile)

global trainDataPath, testDataPath, solutionPath, gloveDir
global NUM_FOLDS, NUM_CLASSES, MAX_NB_WORDS, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM
global BATCH_SIZE, LSTM_DIM, DROPOUT, NUM_EPOCHS, LEARNING_RATE    

trainDataPath = config["train_data_path"]
valDataPath = config["val_data_path"]
testDataPath = config["test_data_path"]
solutionPath = config["solution_path"]
gloveDir = config["glove_dir"]

NUM_FOLDS = config["num_folds"]
NUM_CLASSES = config["num_classes"]
MAX_NB_WORDS = config["max_nb_words"]
MAX_SEQUENCE_LENGTH = config["max_sequence_length"]
EMBEDDING_DIM = config["embedding_dim"]
BATCH_SIZE = config["batch_size"]
LSTM_DIM = config["lstm_dim"]
DROPOUT = config["dropout"]
LEARNING_RATE = config["learning_rate"]
NUM_EPOCHS = config["num_epochs"]

IOError: [Errno 2] No such file or directory: 'testBaseline.config'

In [7]:
print("Processing training data...")
labels, trainTexts = preprocessData(trainDataPath, mode="train")
# Write normalised text to file to check if normalisation works. Disabled now. Uncomment following line to enable   
# writeNormalisedData(trainDataPath, trainTexts)
# print("Processing val data...")
# valIndices, valTexts, vallabels, u1_val, u2_val, u3_val = preprocessData(valDataPath, mode="train")
# print("Processing test data...")
# testIndices, testTexts, u1_test, u2_test, u3_test = preprocessData(testDataPath, mode="test")
# writeNormalisedData(testDataPath, testTexts)
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer(preserve_case=True, reduce_len=True, strip_handles=False)
print("Extracting tokens...")
vocab = []
for sent in u1_train+u2_train+u3_train:
    vocab.extend(tokenizer.tokenize(sent))
    
wordIndex = {} 
for i, word in enumerate(list(set(vocab))):
    wordIndex[word] = i+1

# u1_trainToken, u2_trainToken, u3_trainToken = [tokenizer.tokenize(x) for x in u1_train], [tokenizer.tokenize(x) for x in u2_train], [tokenizer.tokenize(x) for x in u3_train]
# u1_valToken, u2_valToken, u3_valToken = [tokenizer.tokenize(x) for x in u1_val], [tokenizer.tokenize(x) for x in u2_val], [tokenizer.tokenize(x) for x in u3_val]
# u1_testToken, u2_testToken, u3_testToken = [tokenizer.tokenize(x) for x in u1_test], [tokenizer.tokenize(x) for x in u2_test], [tokenizer.tokenize(x) for x in u3_test]

# def text_to_seq(wordIndex, tokens):
#     seq = []
#     for w in tokens:
#         if w in wordIndex:
#             seq.append(wordIndex[w])
#         else:
#             last_key = list(reversed(list(wordIndex)))[0]
#             last_index = wordIndex[last_key]
#             wordIndex[w] = last_index+1
#             seq.append(wordIndex[w])
#     return seq

# u1_trainSequences, u2_trainSequences, u3_trainSequences = [text_to_seq(wordIndex, x) for x in u1_trainToken], [text_to_seq(wordIndex, x) for x in u2_trainToken], [text_to_seq(wordIndex, x) for x in u3_trainToken]
# u1_valSequences, u2_valSequences, u3_valSequences = [text_to_seq(wordIndex, x) for x in u1_valToken], [text_to_seq(wordIndex, x) for x in u2_valToken], [text_to_seq(wordIndex, x) for x in u3_valToken]
# u1_testSequences, u2_testSequences, u3_testSequences = [text_to_seq(wordIndex, x) for x in u1_testToken], [text_to_seq(wordIndex, x) for x in u2_testToken], [text_to_seq(wordIndex, x) for x in u3_testToken]
                                                                                                                                   

Processing training data...


NameError: name 'trainDataPath' is not defined

In [21]:
# wordIndex = tokenizer.word_index
print("Found %s unique tokens." % len(wordIndex))
print("Populating embedding matrix...")
out_of_vocab = []
embeddingMatrix, _ = getEmbeddingMatrix(wordIndex, out_of_vocab)
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                             np.unique(labels),
                                             labels)
print(class_weights)
u1_data = pad_sequences(u1_trainSequences, maxlen=MAX_SEQUENCE_LENGTH)
u2_data = pad_sequences(u2_trainSequences, maxlen=MAX_SEQUENCE_LENGTH)
u3_data = pad_sequences(u3_trainSequences, maxlen=MAX_SEQUENCE_LENGTH)
# labels = to_categorical(np.asarray(labels))
# print("Shape of training data tensor: ", u1_data.shape)
# print("Shape of label tensor: ", labels.shape)

# Randomize data
#np.random.shuffle(trainIndices)

#u1_data = u1_data[trainIndices]
#u2_data = u2_data[trainIndices]
#u3_data = u3_data[trainIndices]

#labels = labels[trainIndices]

# Perform k-fold cross validation
metrics = {"accuracy" : [],
           "microPrecision" : [],
           "microRecall" : [],
           "microF1" : []}

print("Starting k-fold cross validation...")
print('-'*40)
print("Building model...")

Found 14532 unique tokens.
Populating embedding matrix...
Found 1193514 word vectors.
[0.50440737 1.77731559 1.38006029 1.36944444]
Starting k-fold cross validation...
----------------------------------------
Building model...


In [22]:
print(len(out_of_vocab))

1922


In [23]:
print(out_of_vocab)

['phenomennon', 'tensorflow', 'pursnal', 'interestingi', 'golmal', 'weaping', 'meesee', 'cilled', 'timd', 'cbillion', 'twelebrity', 'beera', 'padmavati', 'daznt', 'tiddi', 'propersion', 'e__e', 'aditeya', 'chatbot', 'sunnylenon', 'odrs', 'hoezaay', 'd=<', 'chakkha', '#emojisong', 'veryu', 'chatboard', 'doesnst', 'rddsw', 'ccool', 'understainding', 'heavinly', 'cmplt', 'assked', 'emogi', 'irritater', 'takae', 'denaerys', 'rnda', 'dhapar', 'dhamal', 'whaynyou', 'hanubot', 'heppin', 'getlst', 'lmaorofl', 'sychlops', 'prakrit', 'nicw', 'msgng', '#onekiss', 'd:', 'whatap', 'jesaus', 'complimentsnburns', 'kalsid', 'asach', 'pyrocms', 'qursion', 'half-girlfriend', 'anyonther', 'tyaima', 'beacus', 'useropenreflink', 'dellhi', 'thambs', '::', 'createar', 'lucj', 'wellfare', 'orajinal', 'decreats', 'togethere', 'meesengr', 'marrye', 'peoblem', 'whatodo', 'proosal', 'navratras', 'releaf', 'dindt', 'thjnk', 'vkski', 'libspil', 'loveb', '#bug', 'photodo', 'mbappe', ':/', 'deekila', 'enkoying', 'und

In [50]:
with open('emoji/emoji_ranks.json', 'r', encoding="utf8") as fn:
    e_data = json.load(fn)

pos_emoticons = e_data['pos']
neg_emoticons = e_data['neg']
neutral_emoticons = e_data['neu']
print(len(pos_emoticons))

# pos_emoticons.extend(['(:'])
# neutral_emoticons.extend([''])
# neg_emoticons.extend(['d=<'])

724


In [15]:
l = "im fine 't 12234234 9th 6th 1st 3rd 2nd"
re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", r" <NUMBER> ", l)

"im fine 't  <NUMBER>   <NUMBER> th  <NUMBER> th  <NUMBER> st  <NUMBER> rd  <NUMBER> nd"

In [24]:
import enchant
from enchant.checker import SpellChecker
from nltk.metrics.distance import edit_distance

In [25]:
text = "This is sme text with a fw speling errors in it."

chkr = SpellChecker("en_US", text)
for err in chkr:
    print(err.word + " at position " + str(err.wordpos))
    err.replace(err.word)

t = chkr.get_text()
print("\n" + t)

sme at position 8
fw at position 24
speling at position 27

This is sme text with a fw speling errors in it.


In [26]:
class MySpellChecker():

    def __init__(self, dict_name='en_US', max_dist=2):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = max_dist

    def replace(self, word):
        suggestions = self.spell_dict.suggest(word)

        if suggestions:
            for suggestion in suggestions:
                if edit_distance(word, suggestion) <= self.max_dist:
                    return suggestions[0]

        return word

In [27]:
text = "welcommee"

my_spell_checker = MySpellChecker(max_dist=1)
chkr = SpellChecker("en_US", text)
for err in chkr:
    print(err.word + " at position " + str(err.wordpos))
    err.replace(my_spell_checker.replace(err.word))

t = chkr.get_text()
print("\n" + t)

welcommee at position 0

welcommee


In [35]:
# df = pd.read_csv('tmp.csv', sep=',')
df = pd.read_csv('Sentiment-Analysis-Dataset.csv', sep=',')

In [36]:
df.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [38]:
df.SentimentText.values

(1578624,)